In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 29329531
paper_name = 'bottoms_piotrowski_2018' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [20]:
original_data = pd.read_excel('raw_data/YPDg+2.3%gval.v.YPD+gval.pw.rc.edger.2013_12_01_2 for spotfire 001.xlsx', 
                              sheet_name='YPDg+2.3%gval.v.YPD+gval.pw.rc.')

In [21]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4780 x 6


In [22]:
original_data.head()

,ORF,b.no,foldChange,pvalue,padj,zscoreesque
0,TAX4,2171,4.070313,1.610000e-31,7.670000e-28,-3.221694
1,REI1,2374,8.955910,1.730000e-30,3.230000e-27,0.882946
2,GOS1,4728,0.159774,2.020000e-30,3.230000e-27,-1.571225
3,RCE1,2779,0.118587,2.890000e-29,3.460000e-26,1.177096
4,APA1,1689,3.616984,2.060000e-27,1.970000e-24,-0.924187


In [23]:
original_data['genes'] = original_data['ORF'].astype(str)

In [24]:
# Eliminate all white spaces & capitalize
original_data['genes'] = clean_genename(original_data['genes'])

In [25]:
# Translate to ORFs 
original_data['orfs'] = translate_sc(original_data['genes'], to='orf')

In [26]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orfs'])
print(original_data.loc[~t,])

               ORF  b.no  foldChange        pvalue          padj  zscoreesque  \
index_input                                                                     
79            AAD6  2144    3.187936  8.210000e-15  4.900000e-13    -1.765176   
2520         SDC25  2908    0.822067  2.183395e-01  4.139876e-01    -0.282673   
2558          SDL1  4322    1.162592  2.325143e-01  4.343175e-01     0.217344   
2963         HXT12  1571    1.142491  3.546455e-01  5.719317e-01     0.192183   
4205          FLO8  4193    1.007740  9.657769e-01  1.000000e+00     0.011123   
4360          CRS5    20    1.000000  1.000000e+00  1.000000e+00     0.000000   

             genes   orfs  
index_input                
79            AAD6   AAD6  
2520         SDC25  SDC25  
2558          SDL1   SDL1  
2963         HXT12  HXT12  
4205          FLO8   FLO8  
4360          CRS5   CRS5  


In [27]:
# Untranslated ORFs are all pseudogenes or blocked ORFs
original_data = original_data.loc[t,:]

In [28]:
original_data['data'] = original_data['foldChange']

In [29]:
original_data.set_index('orfs', inplace=True)
original_data.index.name='orf'

In [31]:
original_data = original_data.groupby(original_data.index).mean()

In [32]:
original_data.shape

(4731, 6)

In [33]:
# This dataset (for some reason) contains 573 essential genes -- to remove

In [34]:
essential_orfs = original_data.index.values[is_essential(original_data.index.values)]

Loading SGD features from 2017-04-03.


In [35]:
original_data.drop(index=essential_orfs, inplace=True)

In [37]:
original_data.shape

(4158, 6)

# Prepare the final dataset

In [38]:
data = original_data[['data']].copy()

In [39]:
dataset_ids = [16211]
datasets = datasets.reindex(index=dataset_ids)

In [40]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [41]:
data.head()

dataset_id,16211
data_type,value
orf,
YAL004W,1.497984
YAL005C,1.620630
YAL008W,1.097020
YAL009W,27.742606
YAL010C,0.269592


## Subset to the genes currently in SGD

In [42]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 15


In [43]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [44]:
data.head()

,dataset_id,16211
,data_type,value
gene_id,orf,
1863,YAL004W,1.497984
4,YAL005C,1.620630
6,YAL008W,1.097020
7,YAL009W,27.742606
8,YAL010C,0.269592


# Normalize

In [45]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [46]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [47]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [48]:
data_all.head()


dataset_id           16211          
data_type            value    valuez
gene_id orf                         
1863    YAL004W   1.497984  0.119890
4       YAL005C   1.620630  0.146512
6       YAL008W   1.097020  0.032854
7       YAL009W  27.742606  5.816668
8       YAL010C   0.269592 -0.146751

# Print out

In [49]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [50]:
from IO.save_data_to_db3 import *

In [51]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 29329531...
Inserting the new data...


100%|██████████| 1/1 [00:06<00:00,  6.02s/it]

Updating the data_modified_on field...
